In [1]:
from IPython.core.display import display, HTML
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from astropy.io import ascii

from auxiliary import *
import warnings
import astropy

display(HTML("<style>.container { width:100% !important; }</style>"))
warnings.filterwarnings("ignore", module='astropy.io.votable.tree')
%matplotlib inline

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


# Notebook structure

The notebook consists of four classes:

- catalogueProcessing
    - Used to perform queries and crossmatches
- Astrometry
    - Currently hosts functions for 
        - generating histograms
        - fitting gaussians
    - If HDbscan is implemented, it would go here.
- Photometry
    - Will have methods for making photometric cuts
- Visualization
    - Has methods for plotting, including
        - Plotting single columns
        - Plotting sources overlayed on an image
        - Plotting a gaussian fitted to a histogram generated from
          an arbitrary axis.
    - Will have method for plotting photometric cuts over the appropriate diagram






When catalogProcessing is instantiated, it queries gaia, 2mass, and allwise. This class also provides the ability to query any catalog availible from IRSA. __[More info here.](https://irsa.ipac.caltech.edu/docs/program_interface/TAP.html)__

Parallax for the cluster is optional and has units of milliarcseconds. If given, the radius is assumed to be physical (in pc), otherwise the radius is assumed to be angular (in degrees).

In [2]:
# Define region of interest parameters
ra_cen = "03h47m24s" # hour angle
dec_cen = "+24d07m00s" # degrees
#plx = 1.8481 # milliarcseconds, optional
rad = 0.1 # pc if plx is given, degrees if it is not

#coords = str(ra_cen) + "+" + str(dec_cen)

# Creates an astrio object to read and write tables/log files/etc
io = astrio()

# Instantiates catalogProcessing:
#cat_pr = catalogProcessing(coords,radius=rad)
cat_pr = catalogProcessing(ra_cen, dec_cen, radius=rad, io=io)

# Builds the full data table:
full_table = cat_pr.generate_full_table(gen_small_table=True)
#full_table = Table.read('pleiades_data.dat',format='ascii')

# Instantiates the Visualization and Photometry classes
vis = Visualization(full_table, ra_cen, dec_cen)
phot = Photometry(full_table)

vis.plot_tables('Gaia RA vs DEC',[full_table],('gaia_ra','Gaia RA'),('gaia_dec','Gaia DEC'), squared=True)

# Generates diagnostic error plots:
#  proper motion, right ascension
vis.plot_error('pmra')
#  proper motion, declination
vis.plot_error('pmdec')
#  parallax
vis.plot_error('parallax')

Querying Gaia...
Creating query...
INFO: Query finished. [astroquery.utils.tap.core]
Retrieving results...
Done querying Gaia!

Querying 2MASS...
Created TAP+ (v1.2.1) - Connection:
	Host: irsa.ipac.caltech.edu
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Creating query...
INFO: Query finished. [astroquery.utils.tap.core]
Retrieving results...
Done querying 2MASS!

Querying AllWISE...
Created TAP+ (v1.2.1) - Connection:
	Host: irsa.ipac.caltech.edu
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Creating query...
INFO: Query finished. [astroquery.utils.tap.core]
Retrieving results...
Done querying AllWISE!


Generating full table...
Extracting specified data columns...
Renaming columns...
Crossmatching Gaia with 2MASS...
Crossmatching Gaia with AllWISE...
Crossmatching 2MASS with AllWISE...
Adding objects that do not appear in all catalogs...


TypeError: tuple indices must be integers, not str

Below cell generates a full table consisting of all sources in gaia, 2mass, and allwise at 1pc around the search coordinates (as specified upon instantiation of catalogueProcessing). If a source wasn't crossmatched with a particular catalogue, the column values of this catalogue will be masked.

set gen_small_table to False to return a table with all columns from all three surveys

In [ ]:
# Generates diagnostic plots of ra, dec, parallax, proper motion, and others.
vis.plot_diagnostics()
vis.plot("variability","phot_g_mean_mag",invert_y=True)

Below cell crossmatches the gaia table with the 2mass table. The xmatch radius is determined by looking at the errors of respective match (nearest neighbour) and selecting only objects that are closer than the combined error radii.

The method returns a list of indicies and distances that associates objects in catalogue1 with those in catalogue2.

In [ ]:
#xmatch_res = cat_pr.xmatch(cat_pr.gaia, cat_pr.tmass, ["ra", "dec", "ra_error", "dec_error"], ["ra", "dec", "err_maj", "err_min"])
#print(xmatch_res)

plot_image_overlay gets images from the list of surveys passed to it and plots the sources from the list aux.Visualization was instantiated with, in this case gaia, superimposed on them.

I'll have to do something about overcrowding when there are many sources.

In [ ]:
%matplotlib notebook
vis.plot_images(full_table, ['DSS2 Red', '2MASS-J', 'WISE 4.6', 'WISE 22'], cats_to_plot=["gaia", "allwise"])
#https://astroquery.readthedocs.io/en/latest/skyview/skyview.html

In [ ]:
%matplotlib notebook
vis.plot_hist("parallax", "Parallax")
vis.plot_hist("pmra", "pm RA [mas/yr]")
vis.plot_hist("pmdec", "pm Dec [mas/yr]")

In [ ]:
vis.plot("gaia_ra", "gaia_dec", squared=True, invert_y=True)
vis.plot("pmra", "pmdec",xlim=(-30,30),ylim=(-30,30), squared=True, invert_y=True)
vis.plot("parallax", "parallax_error", xlim=(0,6))


In [ ]:
#%matplotlib inline
#redder, not_redder = phot.keep_reddest()
#vis_redder = Visualization(redder, ra_cen, dec_cen)
#vis_redder.plot_diagnostics()

In [ ]:
#likely_disks, not_likely_disks = phot.keep_likely_disks()
#vis_likely_disks = Visualization(likely_disks, ra_cen, dec_cen)
#vis_likely_disks.plot_diagnostics()

The next cell runs HDBSCAN on the full_table input. It's set to return not more than 20 clusters - arbitrary, I'm still trying to figure out what works for this. It looks to me like the approach of using min_cluster_size may not be effective, but not using it currently gives me in excess of 200 clusters for this dataset, so another solution is needed.

On the plus side, even before applying any photometric cuts, the Pleiades are identified in orange in the plot below, which tells me that the algorithm is robust enough to tolerate some non-ideal behavior in terms of passing it data.

When running the algorithm twice, only removing the expected_clusters parameter for the second run, the number of detected clusters changes (in different ways depending on the dataset). I'm looking into ways to vary this and other parameters dynamically to achieve high-quality results.

In [ ]:
ast = Astrometry(full_table)
candidates_table, cluster_membership_full = ast.identify_clusters(full_table, columns=("pmra","pmdec"), expected_clusters = 20, verbose=True)
candidates_table, cluster_membership_full = ast.identify_clusters(full_table, columns=("pmra","pmdec"), verbose=True)

The cells below apply cuts that select certain parallax ranges. With the parallax constrained, the clustering seems to be much clearer. Again, the Pleiades are identifed, but I think some of the stars identified with the cluster are not in fact part of it. That's part of why I want to incorporate additional columns like parallax in the clustering.